In [219]:
%run 'proof.ipynb'

In [220]:
import numpy as np
import random
import pandas as pd

In [221]:
NUM_PLAYERS = 3
PLAYERS = [i for i in range(1, NUM_PLAYERS+1)]
random.seed(7)  # for reproducible output

# Simulating for one initial state

In [265]:
def give(playerWealth):
    """
    Simulates the give action:
    The minimum of the giver's and receiver's wealth is transferred from the giver to the receiver.
    Returns the updated `playerWealth` list.
    """
    # randomly select a giver and receiver
    players = [i for i in range(NUM_PLAYERS)]
    giver = random.choice(players)
    players.remove(giver)
    receiver = random.choice(players)
    
    amt = min(playerWealth[giver], playerWealth[receiver])
    playerWealth[receiver] += amt
    playerWealth[giver] -= amt
    
    return playerWealth, amt, giver, receiver

In [268]:
def loser(playerWealth):
    """
    Returns the loser (None if no one has lost yet).
    """
    for i in range(NUM_PLAYERS):
        if playerWealth[i] == 0:
            return i
    return None

def reportGameProgress(n, giver, receiver, transferAmt, playerWealth):
    """
    Outputs the game progress.
    - n: an integer representing the number of rounds played so far.
    - giver: the player that is chosen to give.
    - receiver: the player that is chosen to receive.
    - transferAmt: an integer representing the amount of money transferred from the giver to the receiver.
    - playerWealth: a dictionary of player:wealth key-value pairs, representing each player's current wealth.
    """
    print(f"Round {n}:")
    print(f"P{giver+1} gave P{receiver+1} ${transferAmt}")
    print(f"Current stacks: {playerWealth}\n")

In [269]:
def playGame(x,y,z, printProgress=False):
    """
    Plays one game.
    - x, y, z: integers representing the initial wealth of Player 1, 2, 3, respectively.
    - printProgress: if True, prints the game progress.
    Returns the first loser and number of rounds until the game ends.
    """
    # initialise
    stacks = [x,y,z]
    n = 0
    while True:
        stacks, transferAmt, g, r = give(stacks)
        n += 1
        if printProgress:
            reportGameProgress(n, g, r, transferAmt, stacks)
            
        l = loser(stacks)
        if l != None:
            return n, l
    # should not reach this point
    return n, -1

In [271]:
from collections import defaultdict

def h_est(data, x,y,z):
    """
    Returns an estimate of h(n,x,y,z) = Pr(player 1 is eliminated in exactly n rounds), 
    using the game data
    """
    data.sort()
    numGames = len(data)
    hxyz, hyxz = defaultdict(int), defaultdict(int)
    for n, l in data:
        hxyz[n] += (l == 0)
        hyxz[n] += (l == 1)
    
    return {k : v/numGames for k,v in hxyz.items()}, {k : v/numGames for k,v in hyxz.items()}

In [272]:
def f_est(data, x,y,z):
    hxyz, hyxz = h_est(data, x,y,z)
    fxyz = {}
    maxn = max(max(hxyz), max(hyxz))
    
    def dh(n):
        return hxyz.get(n, 0) - hyxz.get(n, 0)
    
    @memoized
    def sumdh(n): 
        return sum([dh(i) for i in range(1,n+1)])
    
    for m in range(2, maxn+1):
        fxyz[m] = sumdh(m) - thresh(m)
        
    return fxyz

In [273]:
from random import randint
def rand_init(V=(lambda x,y,z: x<y and y<z), maxVal=100, maxAttempts=100):
    for i in range(maxAttempts):
        rand = [randint(1, maxVal) for _ in range(NUM_PLAYERS)]
        if V(*rand):
            return rand
    return None

In [274]:
rand_init()

[4, 19, 40]

In [281]:
def sim_xyz(n=100, stat=f_est):
    xyz = rand_init()
    return stat([playGame(*xyz, printProgress=False) for _ in range(n)], *xyz)

In [283]:
sim_xyz(stat=f_est)

{2: 0.10999999999999999,
 3: 0.27,
 4: 0.37,
 5: 0.415,
 6: 0.4275,
 7: 0.44375000000000003,
 8: 0.436875,
 9: 0.4384375,
 10: 0.43921875,
 11: 0.439609375,
 12: 0.4298046875}

# Simulating the game multiple times and estimating probabilities

In [296]:
from statistics import fmean, stdev

def sim(n=10000):
    """
    Simulates the betting game `n` times, with different random initial stacks.
    Returns the average f(n,x,y,z)
    """
    data = [sim_xyz(stat=f_est) for _ in range(n)]
    max_n = max([max(d) for d in data])
    f_avg = {}
    f_std = {}
    f_neg = {}
    for i in range(2, max_n+1):
        fs = [d.get(i, 0) for d in data]
        f_avg[i] = fmean(fs)
        f_std[i] = stdev(fs)
        f_neg[i] = len([f for f in fs if f < 0]) / n  # proportion of non-positives
    return f_avg, f_std, f_neg

In [291]:
sim(n=100000)

({2: -0.00951160000000001,
  3: 0.10252049999999999,
  4: 0.15761050000000001,
  5: 0.184125,
  6: 0.188024175,
  7: 0.157999475,
  8: 0.10751530000000001,
  9: 0.063218496875,
  10: 0.03380515,
  11: 0.017376575000000002,
  12: 0.008671311328125,
  13: 0.00422036875,
  14: 0.002080701953125,
  15: 0.001050271826171875,
  16: 0.0005279693603515625,
  17: 0.0002567925537109375,
  18: 0.0001247981689453125,
  19: 6.039951171875e-05,
  20: 2.6099862670898436e-05,
  21: 9.19997329711914e-06,
  22: 7.5999923706054695e-06},
 {2: 0.10472611489514642,
  3: 0.12122576886380373,
  4: 0.12830398243573382,
  5: 0.1313123063700354,
  6: 0.13580130374193894,
  7: 0.14436197503072867,
  8: 0.14073260086973266,
  9: 0.12004525533681752,
  10: 0.09327096802676471,
  11: 0.06889528806968356,
  12: 0.0493033142762096,
  13: 0.0346148809094887,
  14: 0.024615676322796116,
  15: 0.017684987157066328,
  16: 0.012719354383116636,
  17: 0.00900842574379819,
  18: 0.006319973348400852,
  19: 0.0043886626866460

In [295]:
sim(n=100000)  # <= 0

({2: -0.00925810000000001,
  3: 0.102845,
  4: 0.1578258,
  5: 0.18432695,
  6: 0.188127225,
  7: 0.1579060375,
  8: 0.1073463125,
  9: 0.06347436249999999,
  10: 0.034432990625,
  11: 0.01776090546875,
  12: 0.009191541796875,
  13: 0.0044148130859375,
  14: 0.00225487412109375,
  15: 0.0010860691406250002,
  16: 0.0005562671630859375,
  17: 0.00029289188232421877,
  18: 0.00014359786376953124,
  19: 9.429932861328127e-05,
  20: 4.099983978271484e-05,
  21: 1.709997329711914e-05,
  22: 1.1799992370605468e-05,
  23: 5.899998092651367e-06,
  24: 3.699999523162842e-06},
 {2: 0.104434178549776,
  3: 0.12095108212266092,
  4: 0.12792460475375728,
  5: 0.13086699551606834,
  6: 0.1354198925076085,
  7: 0.14438880693443407,
  8: 0.14080498917488546,
  9: 0.12088476910403707,
  10: 0.09427754363577374,
  11: 0.06984898906646132,
  12: 0.05094671115980283,
  13: 0.03545643136330046,
  14: 0.025522143880711418,
  15: 0.017565947328062652,
  16: 0.0126813088384868,
  17: 0.009230519540105616,
  

In [297]:
sim(n=100000)

({2: -0.00902670000000001,
  3: 0.1031348,
  4: 0.1581853,
  5: 0.18466939999999998,
  6: 0.188148775,
  7: 0.1570627125,
  8: 0.10683098125,
  9: 0.06283335625,
  10: 0.0341936375,
  11: 0.017770865625,
  12: 0.009051499609375001,
  13: 0.004506890625,
  14: 0.00218027802734375,
  15: 0.00106477255859375,
  16: 0.000480270703125,
  17: 0.0002361930419921875,
  18: 0.00012469838256835936,
  19: 6.969954223632813e-05,
  20: 3.2399900817871095e-05,
  21: 1.8899969482421876e-05,
  22: 4.799996185302735e-06,
  23: 1.1999990463256836e-06},
 {2: 0.10487326480627325,
  3: 0.1214586042686376,
  4: 0.12856800986731348,
  5: 0.1315489614361609,
  6: 0.13610761838096055,
  7: 0.14471360918830056,
  8: 0.14050200945859287,
  9: 0.12018086110676576,
  10: 0.09400544413334254,
  11: 0.07011563324973388,
  12: 0.05098870434287367,
  13: 0.036278818067184096,
  14: 0.025179550920504393,
  15: 0.01772897088394027,
  16: 0.011768347346807949,
  17: 0.008268616165280284,
  18: 0.006274638070897127,
  19: